In [35]:
client.models.list().page 

[Model(name='models/embedding-gecko-001', display_name='Embedding Gecko', description='Obtain a distributed representation of a text.', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1024, output_token_limit=1, supported_actions=['embedText', 'countTextTokens'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gemini-1.0-pro-vision-latest', display_name='Gemini 1.0 Pro Vision', description='The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=12288, output_token_limit=4096, supported_actions=['generateContent', 'countTokens'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gem

In [24]:
import util

from API_KEY import GEMINI_API_KEY
import re
import ast
from typing import List, Any

from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

for i in range(1):
    response = client.models.generate_content(
        model="gemma-3-27b-it", contents="""The task is to generate textual descriptions pairs of visual discriminative features to identify whether the central region of an histopathological image patch contains tumor tissue or not. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section.
    Here are the best performing pairs in ascending order. High scores indicate higher quality visual discriminative features.

Current Top 10 prompt pairs:
('Adipose tissue is present, lymphocytes are uniform in size and shape, no mitoses are observed in the lymphocytes, lymphocytes exhibit normal nuclear-cytoplasmic ratio', 'Adipose tissue is replaced by large, atypical tumor cells, tumor cells display significant variability in size and shape (pleomorphism), increased mitotic activity is present within the tumor cells, tumor cells display increased nuclear-cytoplasmic ratio'), Score: 60   
('Adipose tissue is present with small, uniform lymphocytes, lymphocytes exhibit normal nuclear-to-cytoplasmic ratio, no evidence of apoptosis, lymphocytes are uniform in size and shape', 'Adipose tissue is replaced by pleomorphic cells with significant nuclear and cytoplasmic variation, frequent mitotic figures, large, prominent, and irregular nucleoli, tumor cells display significant variability in size and shape (pleomorphism)'), Score: 61   
('Lymphocytes show uniform staining intensity, no significant cellular pleomorphism, nuclei are round and uniform, normal reactive germinal centers are present', 'Tumor cells exhibit variable staining intensity and pleomorphism, cells display large, prominent, and irregular nucleoli, and germinal centers are disrupted'), Score: 62   
('Sinusoidal pattern is preserved, lymphocytes are uniformly sized and shaped, no evidence of necrosis, small, round nuclei', 'Sinusoidal pattern is distorted by tumor, cells exhibit marked pleomorphism in size and shape, presence of large, prominent nucleoli, large, irregular nuclei, frequent mitotic figures'), Score: 63   
('Absence of prominent nucleoli, lymphocytes have smooth nuclear membranes, lymphocytes exhibit normal nuclear-cytoplasmic ratio, no evidence of mitotic activity', 'Presence of large, prominent nucleoli, tumor cells exhibit irregular nuclear membranes, increased mitotic figures observed, significant atypia'), Score: 64   
('No prominent nucleolus is seen, lymphocytes show uniform morphology, lymphocytes exhibit uniform size and shape, no significant increase in mitotic activity', 'Large, prominent, and irregular nucleoli are present, tumor cells grow in sheets or nests, tumor cells display significant pleomorphism, increased mitotic activity, high number of mitotic figures'), Score: 65   
('No significant increase in mitotic activity, small lymphocytes predominate, normal lymphocyte flow, no plasmacytoid differentiation', 'Increased mitotic activity with frequent mitotic figures, large, atypical cells infiltrate, plasmacytic differentiation within the tumor'), Score: 69   
('No prominent nucleolus is seen, lymphocytes are uniform in size and shape, and no atypical cells infiltrating surrounding tissues, lymphocytes have smooth nuclear membranes', 'Large, prominent, and irregular nucleoli are present, tumor cells display significant variability in size and shape, and atypical cells infiltrating surrounding tissues, tumor cells exhibit irregular nuclear membranes'), Score: 73   
('Adipose tissue is present, lymphocytes have smooth nuclear membranes, no atypical cells are identified, normal stromal collagen with delicate fibers, and lymphocytes exhibit a normal CD3/CD20 ratio', 'Adipose tissue is replaced by large, atypical tumor cells, tumor cells exhibit irregular nuclear membranes, significant atypia observed, desmoplastic reaction with dense collagen, and distorted CD3/CD20 ratio indicating clonal expansion'), Score: 82   
('Lymphocytes have uniform staining intensity, lymphocytes exhibit normal nuclear-cytoplasmic ratio, fibrous septa are thin and delicate, no plasmacytoid differentiation', 'Large, irregular cells with prominent nucleoli, thickened and fibrotic septa, increased mitotic activity, plasmacytoid differentiation is prominent within the tumor'), Score: 90 
    
Last 5 iterations improvement: 0
Last Instruction: Write 10 new prompt pairs that are different from the old ones and has a score as high as possible.

Think step by step to write a new clear and concise instruction to improve the score. Output the new instruction in a <instruction> tag.
""")
    print(response.text)

Okay, let's analyze the existing prompt pairs and identify patterns contributing to higher scores.

**Observations:**

*   **Specificity is Key:** Higher-scoring pairs are very specific about *what* is observed (or not observed). They don't just say "atypical cells," but describe *how* they are atypical (pleomorphism, irregular nuclei, prominent nucleoli).
*   **Contrast is Strong:** The best pairs present a clear "normal" vs. "tumor" contrast. The features in the second part of the pair directly counter those in the first.
*   **Nuclear Features Dominate:** Nuclear characteristics (size, shape, staining, nucleoli, mitotic activity) are consistently important discriminators.
*   **Contextual Features Matter:** Features relating to the surrounding tissue (stroma, adipose tissue, sinusoidal pattern, fibrous septa) are also valuable, especially when describing how the tumor *alters* that context.
*   **Immunohistochemical markers:** The highest scoring prompt pair includes CD3/CD20 ratio,

In [26]:
import util

from API_KEY import GEMINI_API_KEY
import re
import ast
from typing import List, Any

from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

for i in range(1):
    response = client.models.generate_content(
        model="gemma-3-27b-it", contents="""The task is to generate textual descriptions pairs of visual discriminative features to identify whether the central region of an histopathological image patch contains tumor tissue or not. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section.
    Here are the best performing pairs in ascending order. High scores indicate higher quality visual discriminative features.

Current Top 10 prompt pairs:
('Adipose tissue is present, lymphocytes are uniform in size and shape, no mitoses are observed in the lymphocytes, lymphocytes exhibit normal nuclear-cytoplasmic ratio', 'Adipose tissue is replaced by large, atypical tumor cells, tumor cells display significant variability in size and shape (pleomorphism), increased mitotic activity is present within the tumor cells, tumor cells display increased nuclear-cytoplasmic ratio'), Score: 60   
('Adipose tissue is present with small, uniform lymphocytes, lymphocytes exhibit normal nuclear-to-cytoplasmic ratio, no evidence of apoptosis, lymphocytes are uniform in size and shape', 'Adipose tissue is replaced by pleomorphic cells with significant nuclear and cytoplasmic variation, frequent mitotic figures, large, prominent, and irregular nucleoli, tumor cells display significant variability in size and shape (pleomorphism)'), Score: 61   
('Lymphocytes show uniform staining intensity, no significant cellular pleomorphism, nuclei are round and uniform, normal reactive germinal centers are present', 'Tumor cells exhibit variable staining intensity and pleomorphism, cells display large, prominent, and irregular nucleoli, and germinal centers are disrupted'), Score: 62   
('Sinusoidal pattern is preserved, lymphocytes are uniformly sized and shaped, no evidence of necrosis, small, round nuclei', 'Sinusoidal pattern is distorted by tumor, cells exhibit marked pleomorphism in size and shape, presence of large, prominent nucleoli, large, irregular nuclei, frequent mitotic figures'), Score: 63   
('Absence of prominent nucleoli, lymphocytes have smooth nuclear membranes, lymphocytes exhibit normal nuclear-cytoplasmic ratio, no evidence of mitotic activity', 'Presence of large, prominent nucleoli, tumor cells exhibit irregular nuclear membranes, increased mitotic figures observed, significant atypia'), Score: 64   
('No prominent nucleolus is seen, lymphocytes show uniform morphology, lymphocytes exhibit uniform size and shape, no significant increase in mitotic activity', 'Large, prominent, and irregular nucleoli are present, tumor cells grow in sheets or nests, tumor cells display significant pleomorphism, increased mitotic activity, high number of mitotic figures'), Score: 65   
('No significant increase in mitotic activity, small lymphocytes predominate, normal lymphocyte flow, no plasmacytoid differentiation', 'Increased mitotic activity with frequent mitotic figures, large, atypical cells infiltrate, plasmacytic differentiation within the tumor'), Score: 69   
('No prominent nucleolus is seen, lymphocytes are uniform in size and shape, and no atypical cells infiltrating surrounding tissues, lymphocytes have smooth nuclear membranes', 'Large, prominent, and irregular nucleoli are present, tumor cells display significant variability in size and shape, and atypical cells infiltrating surrounding tissues, tumor cells exhibit irregular nuclear membranes'), Score: 73   
('Adipose tissue is present, lymphocytes have smooth nuclear membranes, no atypical cells are identified, normal stromal collagen with delicate fibers, and lymphocytes exhibit a normal CD3/CD20 ratio', 'Adipose tissue is replaced by large, atypical tumor cells, tumor cells exhibit irregular nuclear membranes, significant atypia observed, desmoplastic reaction with dense collagen, and distorted CD3/CD20 ratio indicating clonal expansion'), Score: 82   
('Lymphocytes have uniform staining intensity, lymphocytes exhibit normal nuclear-cytoplasmic ratio, fibrous septa are thin and delicate, no plasmacytoid differentiation', 'Large, irregular cells with prominent nucleoli, thickened and fibrotic septa, increased mitotic activity, plasmacytoid differentiation is prominent within the tumor'), Score: 90 
    
Write 10 new prompt pairs that are different from the old ones and has a score as high as possible.
Think step by step to give the output as python code in the format - prompts: list[tuple[negative: str, positive: str]]
""")
    print(response.text)

Okay, let's analyze the existing prompt pairs and try to generate 10 new ones with potentially high scores.

**Analysis of High-Scoring Prompts:**

*   **Key Discriminators:** The highest-scoring prompts consistently focus on these features:
    *   **Pleomorphism:** Variation in cell size and shape is *extremely* important.
    *   **Nucleoli:** Prominent, irregular nucleoli are a strong indicator of tumor cells.
    *   **Mitotic Activity:** Increased mitotic figures are crucial.
    *   **Nuclear-Cytoplasmic Ratio:**  Increased ratio in tumor cells.
    *   **Stroma/Architecture:** Changes in the surrounding tissue (e.g., distorted sinusoids, desmoplastic reaction, thickened septa) are important.
    *   **Immunophenotype:** CD3/CD20 ratio changes are useful.
    *   **Cellular Morphology:** Uniform vs. atypical appearance.
*   **Structure:** The prompts are structured as a clear contrast between normal/benign features and malignant features.
*   **Specificity:** The best prompts us